In [ ]:
import torch
import time
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
torch.set_printoptions(precision=4, sci_mode=False)


 Create a torch Tensor of shape (5, 3) which is filled with zeros. Modify the tensor to set element (0, 2) to 10 and element (2, 0)  to 100.

In [ ]:
t1 = torch.zeros(5,3)
t1.shape

t1[0][2] = 10
t1[2][0] = 100
print(t1)

tensor([[  0.,   0.,  10.],
        [  0.,   0.,   0.],
        [100.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.]])



You have following tensor as input:

```x=torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])```

Using only reshaping functions (like view, reshape, transpose, permute), you need to get at the following tensor as output:

```
tensor([[ 0,  4,  8, 12, 16, 20],
        [ 1,  5,  9, 13, 17, 21],
        [ 2,  6, 10, 14, 18, 22],
        [ 3,  7, 11, 15, 19, 23]])
```



In [ ]:
x=torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])

In [ ]:
tr = x.reshape(6,4)
torch.transpose(tr, 0, 1)

tensor([[ 0,  4,  8, 12, 16, 20],
        [ 1,  5,  9, 13, 17, 21],
        [ 2,  6, 10, 14, 18, 22],
        [ 3,  7, 11, 15, 19, 23]])



Your goal is to measure the speed of linear algebra operations for different levels of vectorization.

1. Construct two matrices $A$ and $B$ with Gaussian random entries of size $1024 \times 1024$. 
1. Compute $C = A B$ using matrix-matrix operations and report the time. (Hint: Use torch.mm)
1. Compute $C = A B$, treating $A$ as a matrix but computing the result for each column of $B$ one at a time. Report the time. (hint use torch.mv inside a for loop)
1. Compute $C = A B$, treating $A$ and $B$ as collections of vectors. Report the time. (Hint: use torch.dot inside nested for loop)

In [ ]:
## Solution 1
torch.manual_seed(42) # dod not chnage this
A =  torch.randn(1024,1024) 
B = torch.randn(1024,1024) 


In [ ]:
## Solution 2
#import time
start=time.time()

C = torch.mm(A, B)

print("Matrix by matrix: " + str(time.time()-start) + " seconds")

Matrix by matrix: 0.035508155822753906 seconds


In [ ]:
## Solution 3
C= torch.empty(1024,1024)
start = time.time()

# code here
for i in range(0,1024):
  torch.mv(A,B[:][i])

print("Matrix by vector: " + str(time.time()-start) + " seconds") 

Matrix by vector: 0.2059786319732666 seconds


In [ ]:
## Solution 4
C= torch.empty(1024,1024)
start = time.time()

# code here
for i in range(0,1024):
  for j in range(0,1024):
    torch.dot(A[i][:],B[:][j])

print("vector by vector: " + str(time.time()-start) + " seconds")  



vector by vector: 9.562596082687378 seconds


 Redo above Question by using GPU 

<font size = 4, color = 'dodgerblue'> **Using GPUs**

How to use GPUs in Google Colab<br>
In Google Colab -- Go to Runtime Tab at top -- select change runtime type -- for hardware accelartor choose GPU

In [ ]:
# Check if GPU is availaible
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [ ]:
## Solution 1
torch.manual_seed(42)
A= torch.randn((1024, 1024),device=device)
B= torch.randn((1024, 1024),device=device)

In [ ]:
## Solution 2
start=time.time()

C = torch.mm(A, B)
    
print("Matrix by matrix: " + str(time.time()-start) + " seconds")

Matrix by matrix: 2.4257400035858154 seconds


In [ ]:
## Solution 3
C= torch.empty(1024,1024, device = device)
start = time.time()

# code here
for i in range(0,1024):
  torch.mv(A,B[:][i])

print("Matrix by vector: " + str(time.time()-start) + " seconds") 

Matrix by vector: 0.035927772521972656 seconds


In [ ]:
## Solution 4
C= torch.empty(1024,1024, device = device)
start = time.time()

# code here
for i in range(0,1024):
  for j in range(0,1024):
    torch.dot(A[i][:],B[:][j])


print("vector by vector: " + str(time.time()-start) + " seconds")  

vector by vector: 22.868422985076904 seconds


** Memory efficient computation**

We want to compute $C \leftarrow A \cdot B + C$, where $A, B$ and $C$ are all matrices. Implement this in the most memory efficient manner. Pay attention to the following two things:

1. Do not allocate new memory for the new value of $C$.
1. Do not allocate new memory for intermediate results if possible. Hint: If you implement this correcly the memory location of C given by id(C) will be same in both the cells below.

In [ ]:
A= torch.randn((1000, 1000),device=device)
B= torch.randn((1000,1000),device=device)
C= torch.randn((1000, 1000),device=device)
print(id(C))

140701140018480


In [ ]:
# your code here
for i in range (0,1000):
  C[i] = torch.dot(A[i], B[i]) + C[i]
print(id(C))

140701140018480


#<font color = 'pickle'>**Broadcast Operations**

1.   List item
2.   List item



In order to perform polynomial fitting we want to compute a design matrix $A$ with 

$$A_{ij} = x_i^j$$

Our goal is to implement this **without a single for loop** entirely using vectorization and broadcast. Here $1 \leq j \leq 3$ and $x = \{1,2,3,4,5\}$. Implement code that generates following A matrix

$$\begin{bmatrix} 1 & 1 & 1 \\ 2 & 4 & 8 \\ 3 & 9 & 27 \\ 4 & 16 & 64 \\ 5 & 25 & 125\end{bmatrix}$$

In [ ]:
# your code here
X= torch.Tensor([[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5]])
X = torch.transpose(X,0,1)
X

t1 = torch.arange(1,4)
t1_mod = t1[None,] 
print(t1_mod)
print(t1_mod.shape)


tensor([[1, 2, 3]])
torch.Size([1, 3])


In [ ]:
t1_mod_rep = t1_mod.repeat(5, 1) 
t1_mod_rep

tensor([[1, 2, 3],
        [1, 2, 3],
        [1, 2, 3],
        [1, 2, 3],
        [1, 2, 3]])

In [ ]:
A = X**t1_mod_rep
print(A)

tensor([[  1.,   1.,   1.],
        [  2.,   4.,   8.],
        [  3.,   9.,  27.],
        [  4.,  16.,  64.],
        [  5.,  25., 125.]])


# <font color = 'pickle'>**Regression from scartch using Pytroch Tensors**</font>

Imagine that you're trying to figure out relationship between two variables x and y . You have some idea but you aren't quite sure yet whether the dependence is linear or quadratic. 

Your goal is to use least mean squares regression to identify the coefficients for the following three models using. The three models are:

1. Quadratic model where $\mathrm{y} = b + w_1 \cdot \mathrm{x} + w_2 \cdot \mathrm{x}^2$.
1. Linear model where $\mathrm{y} = b + w_1 \cdot \mathrm{x}$.
1. Linear model with no bias  where $\mathrm{y} = w_1 \cdot \mathrm{x}$.

- You will use batch gradient descent to estimate the model co-efficients. Batch gradient descent uses complete training data at each iteration. 
- We will implement only training loop (no splitting of data in to training/validation).
- The training loop will have only one for loop. We need to iterate over whole data in each epoch. We do not need to craete batches.
- You may have to try different values of number of epochs/ learning rate to get good results.
- You are not allowed to use Pytorch's nn.module or functions from Pytorch. You will write function for loss function (mean sqaured error), prediction and gradient update from scratch.

## <font color = 'pickle'> **Data**

In [ ]:
x = torch.tensor([[1.5420291, 1.8935232, 2.1603365, 2.5381863, 2.893443, \
                    3.838855, 3.925425, 4.2233696, 4.235571, 4.273397, \
                    4.9332876, 6.4704757, 6.517571, 6.87826, 7.0009003, \
                    7.035741, 7.278681, 7.7561755, 9.121138, 9.728281]])
y = torch.tensor([[63.802246, 80.036026, 91.4903, 108.28776, 122.781975, \
                    161.36314, 166.50816, 176.16772, 180.29395, 179.09758, \
                    206.21027, 272.71857, 272.24033, 289.54745, 293.8488, \
                    295.2281, 306.62274, 327.93243, 383.16296, 408.65967]])

##**Linear Regression With Bias**##

In [ ]:
x = torch.tensor([[1.5420291, 1.8935232, 2.1603365, 2.5381863, 2.893443, \
                    3.838855, 3.925425, 4.2233696, 4.235571, 4.273397, \
                    4.9332876, 6.4704757, 6.517571, 6.87826, 7.0009003, \
                    7.035741, 7.278681, 7.7561755, 9.121138, 9.728281]])
y = torch.tensor([[63.802246, 80.036026, 91.4903, 108.28776, 122.781975, \
                    161.36314, 166.50816, 176.16772, 180.29395, 179.09758, \
                    206.21027, 272.71857, 272.24033, 289.54745, 293.8488, \
                    295.2281, 306.62274, 327.93243, 383.16296, 408.65967]])

In [ ]:
print(x.shape)
x=x.T
y=y.T
print(x.shape)
features = x
labels = y

torch.Size([1, 20])
torch.Size([20, 1])


In [ ]:
def linear_reg(X, w, b):
  """
  Function for linear regression y = Xw + b. 
  Input parameter: features, weights, bias. 
  Output: predicted labels
  """
  return torch.mm(X,w.T) + b

In [ ]:
def mse_loss(ypred, y):
  """
  Squared error loss function.
  Input: actual labels and predicted labels
  Output: squared error loss
  """
  error = ypred - y.view(ypred.shape)
  mean_squared_error = error.T.mm(error)/len(y)
  return mean_squared_error

In [ ]:

w = torch.normal(0, 0.01, size=(1, 1), requires_grad=False)
w1 = torch.normal(0, 0.01, size=(1, 1), requires_grad=False)

w2 = torch.normal(0, 0.01, size=(1, 1), requires_grad=False)
# Create a tensor for bias
b = torch.zeros(1, requires_grad=False)
print(w)

tensor([[-0.0297]])


In [ ]:
import random
def read_data(features, labels):
    """
    Function to access data in mini batches. 
    Input parameters: batch size, features, labels. 
    Output: iterator for minibatch data
    """
    
    # Number of examples in the datasets
    num = len(features)

    # Creating a list of indices from 0 to number of examples in the dataset
    indices = list(range(num))

    # The examples are read at random, in no particular order
    random.shuffle(indices)

    # Return the features and labels of the minibatch
    for i in range(0, num):
        batch_indices = torch.tensor(indices[i:num])
        yield features[batch_indices], labels[batch_indices]

In [ ]:
# weight update step
def sgd(params, params_grad, lr):
    for param, param_grad in zip(params, params_grad):
        param -= lr * param_grad
        

In [ ]:
# Set the values for learning rate and number of epochs
lr = 0.005
epochs = 10

# Iterate over the whole dataset
for epoch in range(epochs):
    
    # Iterate over mini batch
    for X, y in read_data(features,labels):

        # step 1 :forward pass - compute predictions
        ypred = linear_reg(X, w, b)

        # step 2: Calculate minibatch loss
        batch_loss = mse_loss(ypred, y)
        

        # step 3: Compute gradient on loss with respect to weights and bias
        grad_w = 2 *X.T.mm(ypred-y)/len(y)
        grad_b = 2 *(ypred-y).sum()/len(y)

        # step 4: Update parameters using their gradient using optimization algorithm
        sgd([w.T, b], [grad_w, grad_b], lr)  

    
    # Calculate and print loss for the complete epoch
    
    train_l = mse_loss(linear_reg(features, w, b), labels)

    # We can observe the epoch vs loss curve in W&B
    #wandb.log({"/Loss_2_2": train_l.item()})
        
    print(f'epoch {epoch + 1}, loss {float(train_l.item()):f}')

epoch 1, loss 9.765197
epoch 2, loss 10.287228
epoch 3, loss 6.472575
epoch 4, loss 6.009954
epoch 5, loss 5.407040
epoch 6, loss 5.569047
epoch 7, loss 5.098161
epoch 8, loss 4.781171
epoch 9, loss 6.225549
epoch 10, loss 7.268178


##**Linear Regression Wihout Bias**##

In [ ]:
x = torch.tensor([[1.5420291, 1.8935232, 2.1603365, 2.5381863, 2.893443, \
                    3.838855, 3.925425, 4.2233696, 4.235571, 4.273397, \
                    4.9332876, 6.4704757, 6.517571, 6.87826, 7.0009003, \
                    7.035741, 7.278681, 7.7561755, 9.121138, 9.728281]])
y = torch.tensor([[63.802246, 80.036026, 91.4903, 108.28776, 122.781975, \
                    161.36314, 166.50816, 176.16772, 180.29395, 179.09758, \
                    206.21027, 272.71857, 272.24033, 289.54745, 293.8488, \
                    295.2281, 306.62274, 327.93243, 383.16296, 408.65967]])
print(x.shape)
x=x.T
y=y.T
print(x.shape)
x= features
y= labels

torch.Size([1, 20])
torch.Size([20, 1])


In [ ]:
#Linear Regression without bias
def linear_reg_without_bias(X, w):
  return torch.mm(X,w.T) 

In [ ]:
w = torch.normal(0, 0.01, size=(1, 1), requires_grad=False)
w1 = torch.normal(0, 0.01, size=(1, 1), requires_grad=False)

w2 = torch.normal(0, 0.01, size=(1, 1), requires_grad=False)
# Create a tensor for bias
b = torch.zeros(1, requires_grad=False)
print(w)

tensor([[0.0116]])


In [ ]:
import random
def read_data(features, labels):
    """
    Function to access data in mini batches. 
    Input parameters: batch size, features, labels. 
    Output: iterator for minibatch data
    """
    
    # Number of examples in the datasets
    num = len(features)

    # Creating a list of indices from 0 to number of examples in the dataset
    indices = list(range(num))

    # The examples are read at random, in no particular order
    random.shuffle(indices)

    # Return the features and labels of the minibatch
    for i in range(0, num):
        batch_indices = torch.tensor(indices[i:num])
        yield features[batch_indices], labels[batch_indices]

In [ ]:
# weight update step
def sgd(params, params_grad, lr):
    for param, param_grad in zip(params, params_grad):
        param -= lr * param_grad
        

In [ ]:
def mse_loss(ypred, y):
  """
  Squared error loss function.
  Input: actual labels and predicted labels
  Output: squared error loss
  """
  error = ypred - y.view(ypred.shape)
  mean_squared_error = error.T.mm(error)/len(y)
  return mean_squared_error

In [ ]:
# Set the values for learning rate and number of epochs
lr = 0.0001
epochs = 10

# Iterate over the whole dataset
for epoch in range(epochs):
    
    # Iterate over mini batch
    for X, y in read_data(features,labels):

        # step 1 :forward pass - compute predictions
        ypred = linear_reg_without_bias(X, w)

        # step 2: Calculate minibatch loss
        batch_loss = mse_loss(ypred, y)
        

        # step 3: Compute gradient on loss with respect to weights and bias
        grad_w = 2 *X.T.mm(ypred-y)/len(y)
        #grad_b = 2 *(ypred-y).sum()/len(y)

        # step 4: Update parameters using their gradient using optimization algorithm
        sgd([w.T], [grad_w], lr)  

    
    # Calculate and print loss for the complete epoch
    train_l = mse_loss(linear_reg_without_bias(features, w), labels)

    # We can observe the epoch vs loss curve in W&B
    #wandb.log({"/Loss_2_2": train_l.item()})
        
    print(f'epoch {epoch + 1}, loss {float(train_l.item()):f}')

epoch 1, loss 44910.359375
epoch 2, loss 35647.558594
epoch 3, loss 27267.199219
epoch 4, loss 20860.373047
epoch 5, loss 15391.302734
epoch 6, loss 11403.810547
epoch 7, loss 8449.705078
epoch 8, loss 6516.929199
epoch 9, loss 4890.092285
epoch 10, loss 3752.839111






##**Quadratic Term**##



In [ ]:
x = torch.tensor([[1.5420291, 1.8935232, 2.1603365, 2.5381863, 2.893443, \
                    3.838855, 3.925425, 4.2233696, 4.235571, 4.273397, \
                    4.9332876, 6.4704757, 6.517571, 6.87826, 7.0009003, \
                    7.035741, 7.278681, 7.7561755, 9.121138, 9.728281]])
X_squared = torch.square(x)
X_squared

tensor([[ 2.3779,  3.5854,  4.6671,  6.4424,  8.3720, 14.7368, 15.4090, 17.8369,
         17.9401, 18.2619, 24.3373, 41.8671, 42.4787, 47.3105, 49.0126, 49.5016,
         52.9792, 60.1583, 83.1952, 94.6395]])

In [ ]:
x = torch.tensor([[1.5420291, 1.8935232, 2.1603365, 2.5381863, 2.893443, \
                    3.838855, 3.925425, 4.2233696, 4.235571, 4.273397, \
                    4.9332876, 6.4704757, 6.517571, 6.87826, 7.0009003, \
                    7.035741, 7.278681, 7.7561755, 9.121138, 9.728281],  \
                  [ 2.3779,  3.5854,  4.6671,  6.4424,  8.3720, 14.7368, 15.4090, 17.8369,
         17.9401, 18.2619, 24.3373, 41.8671, 42.4787, 47.3105, 49.0126, 49.5016,
         52.9792, 60.1583, 83.1952, 94.6395]
               ])
y = torch.tensor([[63.802246, 80.036026, 91.4903, 108.28776, 122.781975, \
                    161.36314, 166.50816, 176.16772, 180.29395, 179.09758, \
                    206.21027, 272.71857, 272.24033, 289.54745, 293.8488, \
                    295.2281, 306.62274, 327.93243, 383.16296, 408.65967]])
print(x)
x=x.T
y = y.T
print(x.shape)
print(x)
print(y.shape)
print(y)
features= x
labels = y 

tensor([[ 1.5420,  1.8935,  2.1603,  2.5382,  2.8934,  3.8389,  3.9254,  4.2234,
          4.2356,  4.2734,  4.9333,  6.4705,  6.5176,  6.8783,  7.0009,  7.0357,
          7.2787,  7.7562,  9.1211,  9.7283],
        [ 2.3779,  3.5854,  4.6671,  6.4424,  8.3720, 14.7368, 15.4090, 17.8369,
         17.9401, 18.2619, 24.3373, 41.8671, 42.4787, 47.3105, 49.0126, 49.5016,
         52.9792, 60.1583, 83.1952, 94.6395]])
torch.Size([20, 2])
tensor([[ 1.5420,  2.3779],
        [ 1.8935,  3.5854],
        [ 2.1603,  4.6671],
        [ 2.5382,  6.4424],
        [ 2.8934,  8.3720],
        [ 3.8389, 14.7368],
        [ 3.9254, 15.4090],
        [ 4.2234, 17.8369],
        [ 4.2356, 17.9401],
        [ 4.2734, 18.2619],
        [ 4.9333, 24.3373],
        [ 6.4705, 41.8671],
        [ 6.5176, 42.4787],
        [ 6.8783, 47.3105],
        [ 7.0009, 49.0126],
        [ 7.0357, 49.5016],
        [ 7.2787, 52.9792],
        [ 7.7562, 60.1583],
        [ 9.1211, 83.1952],
        [ 9.7283, 94.6395]])
to

In [ ]:
# Importing PyTorch Library
import torch

# Importing random library to generate random dataset
import random

# To get deterministic results
torch.manual_seed(456)
random.seed(123)

In [ ]:
import random
def read_data1(features, labels):
    """
    Function to access data in mini batches. 
    Input parameters: batch size, features, labels. 
    Output: iterator for minibatch data
    """
    
    # Number of examples in the datasets
    num = len(features)

    # Creating a list of indices from 0 to number of examples in the dataset
    indices = list(range(num))

    # The examples are read at random, in no particular order
    random.shuffle(indices)

    # Return the features and labels of the minibatch
    for i in range(0, num):
        batch_indices = torch.tensor(indices[i:num])
        yield features[batch_indices], labels[batch_indices]

In [ ]:
# Call the function read_data and visualize the size of minibatch
for X, y in read_data(features, labels):
    print(X.shape)
    print(y.shape)
    break

torch.Size([20, 2])
torch.Size([20, 1])


In [ ]:
def linear_reg(X, w, b):
  """
  Function for linear regression y = Xw + b. 
  Input parameter: features, weights, bias. 
  Output: predicted labels
  """
  return torch.mm(X,w.T) + b

In [ ]:
def mse_loss(ypred, y):
  """
  Squared error loss function.
  Input: actual labels and predicted labels
  Output: squared error loss
  """
  error = ypred - y.view(ypred.shape)
  mean_squared_error = error.T.mm(error)/len(y)
  return mean_squared_error

In [ ]:
# weight update step
def sgd(params, params_grad, lr):
    for param, param_grad in zip(params, params_grad):
        param -= lr * param_grad

In [ ]:
w = torch.normal(0, 0.01, size=(1, 2), requires_grad=False)

# Create a tensor for bias
b = torch.zeros(1, requires_grad=False)
print(w)

tensor([[-0.0243,  0.0075]])


In [ ]:


# Set the values for learning rate and number of epochs
lr = 0.0001
epochs = 10

# Iterate over the whole dataset
for epoch in range(epochs):
    
    # Iterate over mini batch
    for X, y in read_data1(features,labels):

        # step 1 :forward pass - compute predictions
        ypred = linear_reg(X, w, b)

        # step 2: Calculate minibatch loss
        #batch_loss = mse_loss(ypred, y)
        

        # step 3: Compute gradient on loss with respect to weights and bias
        grad_w = 2 *X.T.mm(ypred-y)/len(y)
        grad_b = 2 *(ypred-y).sum()/len(y)

        # step 4: Update parameters using their gradient using optimization algorithm
        sgd([w.T, b], [grad_w, grad_b], lr)  

    
    # Calculate and print loss for the complete epoch
    train_l = mse_loss(linear_reg(features, w, b), labels)

    # We can observe the epoch vs loss curve in W&B
    #wandb.log({"/Loss_2_2": train_l.item()})
        
    print(f'epoch {epoch + 1}, loss {float(train_l.item()):f}')

epoch 1, loss 7078.635742
epoch 2, loss 3838.150879
epoch 3, loss 3890.479248
epoch 4, loss 3711.339111
epoch 5, loss 3659.002441
epoch 6, loss 4153.273438
epoch 7, loss 3787.488770
epoch 8, loss 3635.017578
epoch 9, loss 3722.775391
epoch 10, loss 5191.581543


##**Conclusion**##

In [ ]:
# Considering the best learning rate and different techniues, best lost is from Linear regression without bias. 
# Sequence: Linear Regression without bias > Linear Regression > Linear Regression with Quadratic Terms 

# <font color = 'pickle'> **Image Classification using - Pixel Similarity**

## <font color = 'pickle'> **Import libraries**

In [ ]:
# import libraries
import torchvision
import torchvision.transforms as transforms
from pathlib import Path
import cv2 as cv 
from google.colab.patches import cv2_imshow # for image display
import pandas as pd

## <font color = 'pickle'> **Mount Google Deive**

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## <font color = 'pickle'>**Path to Dowanload Data**

<font color = 'dodgerblue'>**Make sure you change the Path to where you want to save data.**</font>
In the code below - **data/datasets** is the folder name in my google drive. You can change this to appropriate folder for your drive. For example you may want to save data . In this case the below code should be modified to :

 ```data_folder = Path('/content/drive/MyDrive/')```

In [ ]:
data_folder = Path('/content/drive/MyDrive/Colab Notebooks/MNIST')

## <font color = 'pickle'>**Download MNIST training Data**
The MNIST is a dataset of  handwritten digits, available from this page http://yann.lecun.com/exdb/mnist/. It has a training set of 60,000 examples, and a test set of 10,000 examples. In this task we will use images for digits 3 and 7.

In [ ]:
trainset = torchvision.datasets.MNIST(root = data_folder,
                                      train = True,
                                      download = True,
                                      )

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /content/drive/MyDrive/Colab Notebooks/MNIST/MNIST/raw/train-images-idx3-ubyte.gz to /content/drive/MyDrive/Colab Notebooks/MNIST/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /content/drive/MyDrive/Colab Notebooks/MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to /content/drive/MyDrive/Colab Notebooks/MNIST/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /content/drive/MyDrive/Colab Notebooks/MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to /content/drive/MyDrive/Colab Notebooks/MNIST/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /content/drive/MyDrive/Colab Notebooks/MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/drive/MyDrive/Colab Notebooks/MNIST/MNIST/raw



## <font color = 'pickle'>**Subset of images**

trainset.data has images and trainset.targets has the labels. Below we will create a Boolean mask for digits 3 and 7. We will use this mask to extract subset of images.

In [ ]:
# get the mask 
idx3 = trainset.targets==3
idx7 = trainset.targets==7

In [ ]:
idx3 # This take the value of True wherever the label was 3 and False otherwise

tensor([False, False, False,  ..., False, False, False])

In [ ]:
threes = trainset.data[idx3] # use Boolean mask to extract images for digit three
sevens = trainset.data[idx7] # use Boolean mask to extract images for digit seven

In [ ]:
threes.shape, sevens.shape

(torch.Size([6131, 28, 28]), torch.Size([6265, 28, 28]))

In [ ]:
type(threes[0])

torch.Tensor

In [ ]:
# get a sample image
sample_img_3 = threes[0]

In [ ]:
sample_img_3.shape

torch.Size([28, 28])

### <font color = 'pickle'>**Task1**
Reshape the image so that it has following  shape: 28, 28, 1. Then convert the tensor to numpy array. 

In [ ]:
sample_img_3_numpy =  sample_img_3.reshape([28,28,1]).numpy()

In [ ]:
# let us look at the image
# the above steps were needed because the function cv2_imshow needs a three dimenional numpy array of the 
# shape H x W  X C (height, width, number of channels). The number of channels for black and white images is 1.
cv2_imshow(sample_img_3_numpy)

### <font color = 'pickle'>**Task2**

Repeat the above steps to show the first image for digit 7.
 - Get the first image.
 - Reshape the image to 28, 28, 1
 - Convert the tensor to numpy array


In [ ]:
sample_img_7 = sevens[0]
sample_img_7_numpy =  sample_img_7.reshape([28,28,1]).numpy()
cv2_imshow(sample_img_7_numpy)

## <font color = 'pickle'>**Visualizing images as pixel values**

<font color = 'dodgerblue'> 
- In a computer, images are stored in the form of matrices. The numbers in the matrix are called "pixel values."
- These pixel values show how bright each pixel is.
- 0 means "black," and 255 means "white."
- The matrix of numbers is called the channel, and there is only one channel in a grayscale image.
- For color inages , we have three channels(RGB - Red, Green Blue). That is we need three matrices to store color images.
</font>

In [ ]:
df = pd.DataFrame(sample_img_3[5:24, 7:22].numpy())
df.style.set_properties(**{'font-size':'8pt'})

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,0,0,0,38,43,105,255,253,253,253,253,253,174,6
1,0,0,43,139,224,226,252,253,252,252,252,252,252,252,158
2,0,0,178,252,252,252,252,253,252,252,252,252,252,252,252
3,0,0,109,252,252,230,132,133,132,132,189,252,252,252,252
4,0,0,4,29,29,24,0,0,0,0,14,226,252,252,172
5,0,0,0,0,0,0,0,0,0,0,85,243,252,252,144
6,0,0,0,0,0,0,0,0,0,88,189,252,252,252,14
7,0,0,0,0,0,0,0,91,212,247,252,252,252,204,9
8,0,0,32,125,193,193,193,253,252,252,252,238,102,28,0
9,0,45,222,252,252,252,252,253,252,252,252,177,0,0,0


<font color = 'dodgerblue'> - In the matrix below we have flipped color just for illustartion (We are using 0 for white instead of black).

In [ ]:
df.style.set_properties(**{'font-size':'8pt'}).background_gradient('Greys')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,0,0,0,38,43,105,255,253,253,253,253,253,174,6
1,0,0,43,139,224,226,252,253,252,252,252,252,252,252,158
2,0,0,178,252,252,252,252,253,252,252,252,252,252,252,252
3,0,0,109,252,252,230,132,133,132,132,189,252,252,252,252
4,0,0,4,29,29,24,0,0,0,0,14,226,252,252,172
5,0,0,0,0,0,0,0,0,0,0,85,243,252,252,144
6,0,0,0,0,0,0,0,0,0,88,189,252,252,252,14
7,0,0,0,0,0,0,0,91,212,247,252,252,252,204,9
8,0,0,32,125,193,193,193,253,252,252,252,238,102,28,0
9,0,45,222,252,252,252,252,253,252,252,252,177,0,0,0


### <font color = 'pickle'>**Task3**

<font color = 'pickle'>**Convert threes and sevens generated earlier to float and and take the mean along dimension 0.**

Hint(You can use tensor.float() and then take the mean along dimesnion 0.

Here we are generating an average image by taking the average of each pixel across images.

In [ ]:
mean_threes =  threes.float().mean(0)
mean_sevens =  sevens.float().mean(0)

In [ ]:
mean_threes.shape
mean_sevens.shape

torch.Size([28, 28])

In [ ]:
cv2_imshow(mean_threes.unsqueeze(dim=2).numpy())
print()
cv2_imshow(mean_sevens.unsqueeze(dim=2).numpy())

## <font color = 'pickle'>**Prediction for validation images**

We will calculate the distance of each validation image from the mean_threes and mean_sevens. If the validation image is closest to mean_threes then we will predict 1 else we will predict 0.

## <font color = 'pickle'>**Get Valid Dataset**

In [ ]:
validset = torchvision.datasets.MNIST(root = data_folder,
                                      train = False,
                                      download = True,
                                      )

In [ ]:
# get the mask 
idx3 = validset.targets==3
idx7 = validset.targets==7

In [ ]:
validset.data.shape

torch.Size([10000, 28, 28])

In [ ]:
# get images and labels using the mask
valid_data_3_7 = validset.data[idx3+idx7]
valid_targets_3_7 = validset.targets[idx3 + idx7]

In [ ]:
# check the shape of the inputs (images)
# we have 2038 images with size 28 x 28.
valid_data_3_7.shape

torch.Size([2038, 28, 28])

In [ ]:
# lables for valid dataset
valid_targets_3_7

tensor([7, 7, 3,  ..., 3, 7, 3])

In [ ]:
# change the lable to 1 where the label was 3
valid_targets_3_7[valid_targets_3_7==3] = 1

In [ ]:
# change the lable to 0 where the label was 7
valid_targets_3_7[valid_targets_3_7==7] = 0

In [ ]:
valid_targets_3_7.unique()

tensor([0, 1])

In [ ]:
valid_targets_3_7

tensor([0, 0, 1,  ..., 1, 0, 1])

In [ ]:
valid_data_3_7.shape

torch.Size([2038, 28, 28])

## <font color = 'pickle'>**Distance between images**

### <font color = 'pickle'>**Task4** 

<font color = 'pickle'>**Write a function to calculate the distance between two images a and b.** 
 - Calculate the difference between corresponding pixels of two images.
 - Calculate square of differences.
 - Take the mean (take the mean across last two dimensions (-1, -2) of the square of the differences and then take the square root.

In [ ]:
def dist_images(a, b):
  c = a-b 
  csqrd = torch.square(c)
  cm = torch.mean( csqrd, dim = (-1,-2))
  return torch.sqrt(cm)

In [ ]:
# calculate the distance of the each image in validation set from mean_threes
# bacause of the  broadcasting we are able to use the above function which was written to 
# calculate the distnace between two images

valid_3_dist = dist_images(valid_data_3_7, mean_threes)
valid_3_dist

tensor([70.2972, 71.7095, 79.9791,  ..., 59.1907, 68.7125, 58.1954])

In [ ]:
# calculate the distance of the each image in validation set from mean_sevens
valid_7_dist = dist_images(valid_data_3_7, mean_sevens)

In [ ]:
# if you have done everything right, you will observe following shapes.
valid_3_dist.shape, valid_7_dist.shape

(torch.Size([2038]), torch.Size([2038]))

## <font color = 'pickle'>**Prediction Using Distance**

### <font color = 'pickle'>**Task5**



In [ ]:
# step 1 : generate the tensor filled with zeros taht has the same shape as valid_target_3_7
prediction = torch.zeros([2038])

In [ ]:
# step 2: update the tensor prediction. Update the value to 1, when valid_3_dist < valid_7_dist.
# Hint (Generate a Boolean mask using the condition valid_3_dist < valid_7_dist and use the mask to update value to 1 )
prediction[valid_3_dist<valid_7_dist] =1 


In [ ]:
prediction

tensor([0., 0., 1.,  ..., 1., 0., 1.])

## <font color = 'pickle'>**Accuracy**

In [ ]:
correct = prediction == valid_targets_3_7

In [ ]:
correct

tensor([True, True, True,  ..., True, True, True])

In [ ]:
100 * correct.float().mean().item()

96.6143250465393